In [4]:
import os
import numpy as np
import cv2
import cv2 as ccc
face_cascade = cv2.CascadeClassifier('/usr/share/opencv/haarcascades/haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('/usr/share/opencv/haarcascades/haarcascade_eye.xml')
import matplotlib.pyplot as plt
%matplotlib inline

caffe_root = '~/caffe/' 
import sys
sys.path.insert(0, caffe_root + 'python')
import caffe

plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'


DEMO_DIR = './DemoDir'

categories = [ 'Angry' , 'Disgust' , 'Fear' , 'Happy'  , 'Neutral' ,  'Sad' , 'Surprise']

In [5]:
def detect_emotion(input_image):
    cur_net_dir = 'VGG_S_rgb'
    images_repo = '.'

    mean_filename=os.path.join(DEMO_DIR,cur_net_dir,'mean.binaryproto')
    proto_data = open(mean_filename, "rb").read()
    a = caffe.io.caffe_pb2.BlobProto.FromString(proto_data)
    mean  = caffe.io.blobproto_to_array(a)[0]
    mean = mean.mean(1).mean(1)

    net_pretrained = os.path.join(DEMO_DIR,cur_net_dir,'EmotiW_VGG_S.caffemodel')
    net_model_file = os.path.join(DEMO_DIR,cur_net_dir,'deploy.prototxt')
    VGG_S_Net = caffe.Classifier(net_model_file, net_pretrained,
                           mean=mean,
                           channel_swap=(2,1,0),
                           raw_scale=255,
                           image_dims=(256, 256))

#     input_image = caffe.io.load_image(os.path.join(images_repo,'happy.png'))
    prediction = VGG_S_Net.predict([input_image],oversample=False)
    return 'predicted category is {0}'.format(categories[prediction.argmax()])

In [7]:
cap = cv2.VideoCapture(0)
wait_time = 100
current = 0
while(True):
    current= (current + 1) % wait_time
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)
    temp_face = frame[x:x+w, y:y+h]
    # Display the resulting frame
    cv2.imshow('frame',frame)
    if current == 0:
        ccc.imshow('frame2',temp_face)
        print(detect_emotion(temp_face))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

predicted category is Happy
predicted category is Happy
predicted category is Sad
predicted category is Happy
predicted category is Fear
predicted category is Fear
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Angry
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Fear
predicted category is Happy
predicted category is Happy
predicted category is Happy
predicted category is Fear
predicted category is Angry
predicted category is Happy
predicted category is Fear
predicted category is Fear
predicted category is Happy
predicted category is Happy
predicted category is Fear
predicted category is Fear
